In [ ]:
# ======================================================================
# There are 5 questions in this exam with increasing difficulty from 1-5.
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score significantly
# less than your Category 5 question.
#
# Don't use lambda layers in your model.
# You do not need them to solve the question.
# Lambda layers are not supported by the grading infrastructure.
#
# You must use the Submit and Test button to submit your model
# at least once in this category before you finally submit your exam,
# otherwise you will score zero for this category.
# ======================================================================
#
# NLP QUESTION
#
# Build and train a classifier for the sarcasm dataset.
# The classifier should have a final layer with 1 neuron activated by sigmoid as shown.
# It will be tested against a number of sentences that the network hasn't previously seen
# and you will be scored on whether sarcasm was correctly detected in those sentences.

import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') > 0.95 and logs.get('val_accuracy') > 0.8):
      print("\nDesired accuracy is achieved.")
      self.model.stop_training = True

def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
    urllib.request.urlretrieve(url, 'sarcasm.json')

    # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type='post'
    padding_type='post'
    oov_tok = "<OOV>"
    training_size = 20000

    sentences = []
    labels = []
    # YOUR CODE HERE
    with open("./sarcasm.json", 'r') as f:
      datastore = json.load(f)

    for item in datastore:
      sentences.append(item['headline'])
      labels.append(item['is_sarcastic'])

    training_sentences = sentences[0:training_size]
    testing_sentences = sentences[training_size:]
    training_labels = labels[0:training_size]
    testing_labels = labels[training_size:]

    tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
    tokenizer.fit_on_texts(training_sentences)
    word_index = tokenizer.word_index

    training_sequences = tokenizer.texts_to_sequences(training_sentences)
    training_padded = pad_sequences(training_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)
    testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
    testing_padded = pad_sequences(testing_sequences, maxlen = max_length, padding = padding_type, truncating = trunc_type)
    
    training_labels = np.array(training_labels)
    testing_labels = np.array(testing_labels)

    model = tf.keras.Sequential([
    # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(64, 5, activation = 'relu'),
        tf.keras.layers.MaxPooling1D(pool_size = 4),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    callback = myCallback()
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = 'adam',
        metrics = ['accuracy'])
    
    model.fit(
        training_padded,
        training_labels,
        epochs = 100,
        validation_data = (
            testing_padded,
            testing_labels),
        callbacks = callback)
    
    return model

# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, your saved .h5 model will
# be sent to the testing infrastructure for scoring
# and the score will be returned to you.
if __name__ == '__main__':
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/100
625/625 [==============================] - 6s 7ms/step - loss: 0.6174 - accuracy: 0.6489 - val_loss: 0.5714 - val_accuracy: 0.7011
Epoch 2/100
625/625 [==============================] - 4s 6ms/step - loss: 0.5554 - accuracy: 0.7505 - val_loss: 0.5463 - val_accuracy: 0.7591
Epoch 3/100
625/625 [==============================] - 4s 6ms/step - loss: 0.5819 - accuracy: 0.7037 - val_loss: 0.6851 - val_accuracy: 0.5633
Epoch 4/100
625/625 [==============================] - 4s 6ms/step - loss: 0.6865 - accuracy: 0.5603 - val_loss: 0.6894 - val_accuracy: 0.5633
Epoch 5/100
625/625 [==============================] - 4s 6ms/step - loss: 0.6867 - accuracy: 0.5603 - val_loss: 0.6851 - val_accuracy: 0.5633
Epoch 6/100
625/625 [==============================] - 4s 6ms/step - loss: 0.6867 - accuracy: 0.5603 - val_loss: 0.6851 - val_accuracy: 0.5633
Epoch 7/100
625/625 [==============================] - 4s 6ms/step - loss: 0.6550 - accuracy: 0.6025 - val_loss: 0.5083 - val_accuracy: 0.7727